In [1]:
import torch
import torch.nn as nn
import pdb

import os
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('/mnt/fyp')  
from preprocessing.make_dataset import GestureDataset

In [3]:
# add Gaussian noise on test dataset (tensor)
def add_noise(img, noise_std):
    noise = torch.randn(img.shape[0], 3, 224, 224) * noise_std
    noisy_img = img + noise
    noisy_img = torch.clamp(noisy_img, 0, 255)
    return noisy_img

In [4]:
device = torch.device("cuda:0")

In [34]:
# load val_loader
loader = torch.load("/mnt/fyp/data/val_loader.pt")

# load trained model
model = torch.load('/mnt/fyp/models/img_teng_imu_model.pth') # change to img_teng_imu_model.pth / img_model.pth

# eval
model.eval()

noise_std = 10 # change to 0/10/20/30/40/50

for idx,batch in enumerate(loader):
    img = batch['data'][:,:3*224*224].reshape(-1,3,224,224)
    
    # make noisy val_imgs
    noisy_img = add_noise(img, noise_std).to(device)
    
    teng = batch['data'][:,224*224*3:224*224*3+500].reshape(-1,50,10).to(device)
    
    imu = batch['data'][:,224*224*3+500:].reshape(-1,50,18).to(device) 
    
    label = batch['label'].to(torch.long).to(device).squeeze()
    
    if idx==0:
        gt = label
    else:
        gt = torch.cat((gt, label), dim=0)
    
    outputs = model(noisy_img, teng, imu) # for img_teng_imu_model
    #outputs = model(noisy_img) # for img_model
    
    _, predicted = torch.max(outputs, 1) 
    
    if idx==0:
        predict = predicted
    else:
        predict = torch.cat((predict, predicted), dim=0)

#torch.save(img_data,"/mnt/fyp/noise/data/noisy_img_data_std10.pt")

In [35]:
# print val_accuracy

val_res = (gt == predict).tolist()

val_acc = sum(1 for item in val_res if item == True) / len(val_res)

print(val_acc)

0.9987179487179487
